# EDA


- The os module has a perfect method to list files in a directory.
- Pandas json normalize could work here but is not necessary to convert the JSON data to a dataframe.
- You may need a nested for-loop to access each sale!
- We've put a lot of time into creating the structure of this repository, and it's a good example for future projects.  In the file functions_variables.py, there is an example function that you can import and use.  If you have any variables, functions or classes that you want to make, they can be put in the functions_variables.py file and imported into a notebook.  Note that only .py files can be imported into a notebook. If you want to import everything from a .py file, you can use the following:
```python
from functions_variables import *
```
If you just import functions_variables, then each object from the file will need to be prepended with "functions_variables"\
Using this .py file will keep your notebooks very organized and make it easier to reuse code between notebooks.

In [75]:
# (this is not an exhaustive list of libraries)
import pandas as pd
import numpy as np
import os
import json
from pprint import pprint
from functions_variables import encode_tags

## Data Importing

In [95]:
# Directory containing JSON files
directory = r"C:\Users\lai29\OneDrive - UBC\Documents\GitHub\Midterm-project\data"

# Initialize an empty list to store data
data_list = []

# Loop through each JSON file in the directory
for filename in os.listdir(directory):
    if filename.endswith(".json"):
        file_path = os.path.join(directory, filename)
        try:
            with open(file_path, "r", encoding="utf-8") as file:
                data = json.load(file)  # Load JSON data

            # Check if expected keys exist before accessing them
            if isinstance(data, dict) and "data" in data and isinstance(data["data"], dict) and "results" in data["data"]:
                for record in data["data"]["results"]:
                    if isinstance(record, dict):  # Ensure record is a dictionary before processing
                        flat_record = record.copy()

                        # Extract state and city from location
                        flat_record["state"] = None
                        flat_record["city"] = None

                        if "location" in record and isinstance(record["location"], dict):
                            address = record["location"].get("address", {})
                            flat_record["state"] = address.get("state", None)
                            flat_record["city"] = address.get("city", None)

                        # Flatten description
                        if "description" in record and isinstance(record["description"], dict):
                            flat_record.update(record["description"])
                            del flat_record["description"]

                        # Flatten products (dictionary)
                        if "products" in record and isinstance(record["products"], dict):
                            flat_record.update(pd.json_normalize(record["products"]).to_dict(orient="records")[0])
                            del flat_record["products"]

                        # Append flattened record to list
                        data_list.append(flat_record)

        except json.JSONDecodeError as e:
            print(f"JSON Error in file {filename}: {e}")

# Convert list of dictionaries into a DataFrame
df_combined = pd.DataFrame(data_list)

# Display or save the combined DataFrame
df_combined.to_csv("combined_data.csv", index=False)

    state    city
0  Alaska  Juneau
1  Alaska  Juneau
2  Alaska  Juneau
3  Alaska  Juneau
4  Alaska  Juneau


In [96]:
print(df_combined.head())

primary_photo      last_update_date  \
0  {'href': 'https://ap.rdcpix.com/07097d34c98a59...  2023-09-19T20:52:50Z   
1                                               None                  None   
2                                               None                  None   
3                                               None                  None   
4                                               None                  None   

                                              source  \
0  {'plan_id': None, 'agents': [{'office_name': '...   
1                                               None   
2                                               None   
3                                               None   
4                                               None   

                                                tags  \
0  [carport, community_outdoor_space, cul_de_sac,...   
1                                               None   
2                                               None   
3                                               None   
4                                               None   

                                           permalink status  \
0       9453-Herbert-Pl_Juneau_AK_99801_M90744-30767   sold   
1  8477-Thunder-Mountain-Rd_Juneau_AK_99801_M9424...   sold   
2      4515-Glacier-Hwy_Juneau_AK_99801_M94790-68516   sold   
3  17850-Point-Stephens-Rd_Juneau_AK_99801_M98793...   sold   
4  9951-Stephen-Richards-Memorial-Dr_Juneau_AK_99...   sold   

                     list_date open_houses  \
0  2023-06-29T21:16:25.000000Z        None   
1                         None        None   
2                         None        None   
3                         None        None   
4                         None        None   

                                            branding  list_price  ...    sqft  \
0  [{'name': 'EXP Realty LLC - Southeast Alaska',...    554950.0  ...  1821.0   
1  [{'name': None, 'photo': None, 'type': 'Office'}]         NaN  ...     NaN   
2  [{'name': None, 'photo': None, 'type': 'Office'}]         NaN  ...     NaN   
3  [{'name': None, 'photo': None, 'type': 'Office'}]         NaN  ...     NaN   
4  [{'name': None, 'photo': None, 'type': 'Office'}]         NaN  ...     NaN   

  baths sub_type baths_1qtr garage stories beds           type    brand_name  \
0   2.0     None       None    1.0     NaN  3.0  single_family  basic_opt_in   
1   NaN     None       None    NaN     NaN  NaN           None           NaN   
2   NaN     None       None    NaN     NaN  NaN           None           NaN   
3   NaN     None       None    NaN     NaN  NaN           None           NaN   
4   NaN     None       None    NaN     NaN  NaN           None           NaN   

  products  
0      NaN  
1      NaN  
2      NaN  
3      NaN  
4      NaN  

[5 rows x 41 columns]

## Data Cleaning and Wrangling

At this point, ensure that you have all sales in a dataframe.
- Take a quick look at your data (i.e. `.info()`, `.describe()`) - what do you see?
- Is each cell one value, or do some cells have lists?
- What are the data types of each column?
- Some sales may not actually include the sale price (target).  These rows should be dropped.
- There are a lot of NA/None values.  Should these be dropped or replaced with something?
    - You can drop rows or use various methods to fills NA's - use your best judgement for each column 
    - i.e. for some columns (like Garage), NA probably just means no Garage, so 0
- Drop columns that aren't needed
    - Don't keep the list price because it will be too close to the sale price. Assume we want to predict the price of houses not yet listed

In [ ]:
# load and concatenate data here
# drop or replace values as necessary

In [94]:
df_combined.info()

Index(['list_date', 'open_houses', 'flags', 'sold_date', 'name', 'sub_type',
       'baths_1qtr', 'type', 'brand_name'],
      dtype='object')

In [ ]:
df_combined.describe()

In [ ]:
df_combined.dtypes

#Create a list of categorical variables
categorical = df_combined.select_dtypes(include=['object']).columns

#Create a list of numerical variables
numerical = df_combined.select_dtypes(exclude=['object']).columns

print(categorical)
print(numerical)

In [ ]:
#Columns to drop

columns_to_drop = ['primary_photo', 'last_update_date', 'source', 'permalink','status','branding','lead_attributes' ,'property_id', 'photos', 'listing_id','other_listings', 'community', 'products',
       'virtual_tours','open_houses','location','flags']

In [ ]:
df_dropped = df_combined.drop(columns=columns_to_drop)
df_dropped.head()

### Dealing with Tags

Consider the fact that with tags, there are a lot of categorical variables.
- How many columns would we have if we OHE tags, city and state?
- Perhaps we can get rid of tags that have a low frequency.

In [ ]:
# OHE categorical variables/ tags here
# tags will have to be done manually

def encode_tags(df):
    '''
    Creation of a function that would one-hot encode the 'tags' column of a dataframe
    '''
    """
    One-hot encode the 'tags' column.

    Args:
        df (pandas.DataFrame): The DataFrame containing a 'tags' column.

    Returns:
        pandas.DataFrame: Modified with encoded tag columns.
    """
    return df.join(df["tags"].str.get_dummies(sep=", ")).drop(columns=["tags"])

In [ ]:
df_encoded = encode_tags(df_dropped)
df_encoded.head()

In [ ]:
#Testing fill NA for price reduced amount
df_encoded['price_reduced_amount'].fillna(0, inplace=True)
df_encoded.shape

### Dealing with Cities

- Sales will vary drastically between cities and states.  Is there a way to keep information about which city it is without OHE?
- Could we label encode or ordinal encode?  Yes, but this may have undesirable effects, giving nominal data ordinal values.
- What we can do is use our training data to encode the mean sale price by city as a feature (a.k.a. Target Encoding)
    - We can do this as long as we ONLY use the training data - we're using the available data to give us a 'starting guess' of the price for each city, without needing to encode city explicitly
- If you replace cities or states with numerical values (like the mean price), make sure that the data is split so that we don't leak data into the training selection. This is a great time to train test split. Compute on the training data, and join these values to the test data
- Note that you *may* have cities in the test set that are not in the training set. You don't want these to be NA, so maybe you can fill them with the overall mean

In [ ]:
# perform train test split here
# do something with state and city

## Extra Data - STRETCH

> This doesn't need to be part of your Minimum Viable Product (MVP). We recommend you write a functional, basic pipeline first, then circle back and join new data if you have time

> If you do this, try to write your downstream steps in a way it will still work on a dataframe with different features!

- You're not limited to just using the data provided to you. Think/ do some research about other features that might be useful to predict housing prices. 
- Can you import and join this data? Make sure you do any necessary preprocessing and make sure it is joined correctly.
- Example suggestion: could mortgage interest rates in the year of the listing affect the price? 

In [ ]:
# import, join and preprocess new data here

## EDA/ Visualization

Remember all of the EDA that you've been learning about?  Now is a perfect time for it!
- Look at distributions of numerical variables to see the shape of the data and detect outliers.    
    - Consider transforming very skewed variables
- Scatterplots of a numerical variable and the target go a long way to show correlations.
- A heatmap will help detect highly correlated features, and we don't want these.
    - You may have too many features to do this, in which case you can simply compute the most correlated feature-pairs and list them
- Is there any overlap in any of the features? (redundant information, like number of this or that room...)

In [ ]:
# perform EDA here

## Scaling and Finishing Up

Now is a great time to scale the data and save it once it's preprocessed.
- You can save it in your data folder, but you may want to make a new `processed/` subfolder to keep it organized